In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!nvidia-smi

Thu Jun 26 01:42:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
!unzip '/content/drive/MyDrive/Facial Emotion/data.zip' -d '/content/drive/MyDrive/Facial Emotion/'

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/Facial Emotion/data/train/Sad/370.jpg  
  inflating: /content/drive/MyDrive/Facial Emotion/data/train/Sad/3700.jpg  
  inflating: /content/drive/MyDrive/Facial Emotion/data/train/Sad/3701.jpg  
  inflating: /content/drive/MyDrive/Facial Emotion/data/train/Sad/3702.jpg  
  inflating: /content/drive/MyDrive/Facial Emotion/data/train/Sad/3703.jpg  
  inflating: /content/drive/MyDrive/Facial Emotion/data/train/Sad/3704.jpg  
  inflating: /content/drive/MyDrive/Facial Emotion/data/train/Sad/3705.jpg  
  inflating: /content/drive/MyDrive/Facial Emotion/data/train/Sad/3706.jpg  
  inflating: /content/drive/MyDrive/Facial Emotion/data/train/Sad/3707.jpg  
  inflating: /content/drive/MyDrive/Facial Emotion/data/train/Sad/3708.jpg  
  inflating: /content/drive/MyDrive/Facial Emotion/data/train/Sad/3709.jpg  
  inflating: /content/drive/MyDrive/Facial Emotion/data/train/Sad/371.jpg  
  inflating: /content/drive

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
import os


train_data_dir='/content/drive/MyDrive/Facial Emotion/data/train'
validation_data_dir='/content/drive/MyDrive/Facial Emotion/data/test'


train_datagen = ImageDataGenerator(
                                        rescale=1./255,
                                        rotation_range=30,
                                        shear_range=0.3,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                                        train_data_dir,
                                        color_mode='grayscale',
                                        target_size=(48, 48),
                                        batch_size=32,
                                        class_mode='categorical',
                                        shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                                        validation_data_dir,
                                                        color_mode='grayscale',
                                                        target_size=(48, 48),
                                                        batch_size=32,
                                                        class_mode='categorical',
                                                        shuffle=True)


class_labels=['Angry','Disgust', 'Fear', 'Happy','Neutral','Sad','Surprise']

img, label = train_generator.__next__()


model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax'))

model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


train_path = "/content/drive/MyDrive/Facial Emotion/data/train"
test_path = "/content/drive/MyDrive/Facial Emotion/data/test"

num_train_imgs = 0
for root, dirs, files in os.walk(train_path):
    num_train_imgs += len(files)

num_test_imgs = 0
for root, dirs, files in os.walk(test_path):
    num_test_imgs += len(files)

print(num_train_imgs)
print(num_test_imgs)
epochs=30

history=model.fit(train_generator,
                steps_per_epoch=num_train_imgs//32,
                epochs=epochs,
                validation_data=validation_generator,
                validation_steps=num_test_imgs//32)

model.save('/content/drive/MyDrive/Facial Emotion/model_file_30epochs.h5')

Found 22629 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 44, 44, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 20, 20, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,489,095 (9.50 MB)

...

707/707 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.5625 - loss: 1.1600 - val_accuracy: 0.4760 - val_loss: 1.5669
Epoch 27/30
707/707 ━━━━━━━━━━━━━━━━━━━━ 132s 112ms/step - accuracy: 0.6321 - loss: 1.0220 - val_accuracy: 0.4665 - val_loss: 1.7517
Epoch 28/30
  1/707 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.6250 - loss: 1.0030

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


707/707 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.6250 - loss: 1.0030 - val_accuracy: 0.4662 - val_loss: 1.7595
Epoch 29/30
707/707 ━━━━━━━━━━━━━━━━━━━━ 132s 111ms/step - accuracy: 0.6354 - loss: 1.0201 - val_accuracy: 0.4816 - val_loss: 1.7420
Epoch 30/30
  1/707 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.5312 - loss: 1.3995

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


707/707 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.5312 - loss: 1.3995 - val_accuracy: 0.4785 - val_loss: 1.7447


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
import os


train_data_dir='/content/drive/MyDrive/Facial Emotion/data/train'
validation_data_dir='/content/drive/MyDrive/Facial Emotion/data/test'


train_datagen = ImageDataGenerator(
                                        rescale=1./255,
                                        rotation_range=30,
                                        shear_range=0.3,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                                        train_data_dir,
                                        color_mode='grayscale',
                                        target_size=(48, 48),
                                        batch_size=32,
                                        class_mode='categorical',
                                        shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                                        validation_data_dir,
                                                        color_mode='grayscale',
                                                        target_size=(48, 48),
                                                        batch_size=32,
                                                        class_mode='categorical',
                                                        shuffle=True)


class_labels=['Angry','Disgust', 'Fear', 'Happy','Neutral','Sad','Surprise']

img, label = train_generator.__next__()


model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax'))

model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


train_path = "/content/drive/MyDrive/Facial Emotion/data/train"
test_path = "/content/drive/MyDrive/Facial Emotion/data/test"

num_train_imgs = 0
for root, dirs, files in os.walk(train_path):
    num_train_imgs += len(files)

num_test_imgs = 0
for root, dirs, files in os.walk(test_path):
    num_test_imgs += len(files)

print(num_train_imgs)
print(num_test_imgs)
epochs=100

history=model.fit(train_generator,
                steps_per_epoch=num_train_imgs//32,
                epochs=epochs,
                validation_data=validation_generator,
                validation_steps=num_test_imgs//32)

model.save('/content/drive/MyDrive/Facial Emotion/model_file_30epochs.h5')

Found 22629 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 44, 44, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 20, 20, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,489,095 (9.50 MB)

...

707/707 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.7812 - loss: 0.7968 - val_accuracy: 0.5218 - val_loss: 1.6903
Epoch 97/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 142s 130ms/step - accuracy: 0.6797 - loss: 0.8733 - val_accuracy: 0.5084 - val_loss: 1.6177
Epoch 98/100
  1/707 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.6875 - loss: 0.8448

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


707/707 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.6875 - loss: 0.8448 - val_accuracy: 0.5081 - val_loss: 1.6198
Epoch 99/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 83s 117ms/step - accuracy: 0.6870 - loss: 0.8719 - val_accuracy: 0.5117 - val_loss: 1.6974
Epoch 100/100
  1/707 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - accuracy: 0.6250 - loss: 0.9368

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


707/707 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.6250 - loss: 0.9368 - val_accuracy: 0.5081 - val_loss: 1.6933
